In [30]:
import pandas as pd
import numpy as np
import glob
import os

In [173]:
# read in the summary file, skipping the first 9 rows which contain document-level notes
summary = pd.read_csv('/Users/ericaspada/Desktop/classes/Database systems/E-GEOD-47962.sdrf.rtf',
                   skiprows=9,
                  sep='\t')

# column 'Derived Array Data File' corresponds to the individual data files in E-GEOD-47962

# read the filenames and filenames with full path from the folder E-GEOD-47962
filenames = os.listdir('/Users/ericaspada/Desktop/classes/Database systems/E-GEOD-47962')
filenames_path = glob.glob('/Users/ericaspada/Desktop/classes/Database systems/E-GEOD-47962/*txt')

filenames.remove('.DS_Store')

# iterate through the files in E-GEOD-47962
# transpose the dataframes and append them to a list of dataframes
dfs = []
for i in filenames_path:
    df = pd.read_csv(i,sep='\t')
    df.name = i
    df.set_index('Reporter Identifier')
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df[1:]
    dfs.append(df)

# iterate through the list of dataframes and append the rows to the first
df_master = dfs[0]
for i in dfs[1:]:
    df_master = df_master.append(i)

# add a column for the filenames
df_master['filename'] = filenames

# merge the summary dataframe with df_master on the filename
combined = summary.merge(df_master, how='left', left_on='Derived Array Data File', right_on='filename')

In [176]:
# write combined df to text file
with open('/Users/ericaspada/Desktop/classes/Database systems/E-GEOD-47962/combined.txt','w') as outfile:
    combined.to_string(outfile)
    
# and to csv
with open('/Users/ericaspada/Desktop/classes/Database systems/E-GEOD-47962/combined.csv','w') as outfile:
    combined.to_csv(outfile, sep=',')